In [1]:
import math
import numpy
from matplotlib import pyplot 
%matplotlib inline

In [3]:
#establish grid 
N = 51 
x_start, x_end = -1.0, 2.0
y_start, y_end = -0.5, 0.5
x = numpy.linspace(x_start, x_end, N)
y = numpy.linspace(y_start, y_end, N)
X, Y = numpy.meshgrid(x, y)

In [5]:
#load data files

NameError: name 'NACA0012_x' is not defined